In [1]:
!nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-bbf29328-2871-9f08-d09c-1d6f513cdf61)


In [2]:
import pathlib
import numpy as np
data_dir = pathlib.Path("/content/drive/MyDrive/AI/Detector/images")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
print(class_names)

['Adina' 'Bibanu' 'Papagalu' 'Pelicanu']


In [3]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2


In [ ]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders
splitfolders.ratio("/content/drive/MyDrive/AI/Detector/images",
output="/content/drive/MyDrive/AI/Detector/Image_split", seed=1337, ratio=(.7, .2, .1))

Copying files: 627 files [00:43, 14.39 files/s]


In [ ]:
for folder in ['train', 'test', 'val']:
    for file in os.listdir(os.path.join('/content/drive/MyDrive/AI/Detector/Image_split', folder, 'Adina')):
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('/content/drive/MyDrive/AI/Detector', 'labels', filename)
        if os.path.exists(existing_filepath):
            new_filepath = os.path.join('/content/drive/MyDrive/AI/Detector/Image_split', folder, 'labels', filename)
            os.replace(existing_filepath, new_filepath)

In [ ]:
for folder in ['train', 'test', 'val']:
    for file in os.listdir(os.path.join('/content/drive/MyDrive/AI/Detector/Image_split', folder, 'Bibanu')):
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('/content/drive/MyDrive/AI/Detector', 'labels', filename)
        if os.path.exists(existing_filepath):
            new_filepath = os.path.join('/content/drive/MyDrive/AI/Detector/Image_split', folder, 'labels', filename)
            os.replace(existing_filepath, new_filepath)

In [ ]:
for folder in ['train', 'test', 'val']:
    for file in os.listdir(os.path.join('/content/drive/MyDrive/AI/Detector/Image_split', folder, 'Papagalu')):
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('/content/drive/MyDrive/AI/Detector', 'labels', filename)
        if os.path.exists(existing_filepath):
            new_filepath = os.path.join('/content/drive/MyDrive/AI/Detector/Image_split', folder, 'labels', filename)
            os.replace(existing_filepath, new_filepath)

In [ ]:
for folder in ['train', 'test', 'val']:
    for file in os.listdir(os.path.join('/content/drive/MyDrive/AI/Detector/Image_split', folder, 'Pelicanu')):
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('/content/drive/MyDrive/AI/Detector', 'labels', filename)
        if os.path.exists(existing_filepath):
            new_filepath = os.path.join('/content/drive/MyDrive/AI/Detector/Image_split', folder, 'labels', filename)
            os.replace(existing_filepath, new_filepath)

In [4]:
import albumentations as alb

In [5]:
augumentation = alb.Compose([alb.RandomCrop(width=450, height=450),
                             alb.HorizontalFlip(p=0.5),
                             alb.RandomBrightnessContrast(p=0.2),
                             alb.RandomGamma(p=0.2),
                             alb.RGBShift(p=0.2),
                             alb.VerticalFlip(p=0.6)],
                            bbox_params=alb.BboxParams(format='albumentations',
                                                       label_fields=['class_labels']))

In [ ]:
Image_Split = r'/content/drive/MyDrive/AI/Detector/Image_split'
aug_data = r'/content/drive/MyDrive/AI/Detector/aug_data'

In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join(Image_Split, partition, 'Adina')):
        img = cv2.imread(os.path.join(Image_Split, partition, 'Adina', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join(Image_Split, partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
                
            width, heigth = img.shape[:2]
            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [width,heigth,width,heigth]))

        try: 
            for x in range(30):
                augmented = augumentation(image=img, bboxes=[coords], class_labels=['Adina'])
                cv2.imwrite(os.path.join(aug_data, partition, 'Adina', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join(aug_data, partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

Expected y_max for bbox [0.0069444444444444284, 0.018518518518518517, 0.749375, 1.2835663533624257, 'Adina'] to be in the range [0.0, 1.0], got 1.2835663533624257.
Expected y_max for bbox [0.25670138888888894, 0.4109494824443756, 0.5379513888888889, 1.0265139791602187, 'Adina'] to be in the range [0.0, 1.0], got 1.0265139791602187.
Expected y_max for bbox [0.2569673184292289, 0.6112134502923977, 0.589856799121639, 1.045423976608187, 'Adina'] to be in the range [0.0, 1.0], got 1.045423976608187.
Expected y_max for bbox [0.47916666666666663, 0.5895061728395061, 0.7222222222222223, 1.0570987654320985, 'Adina'] to be in the range [0.0, 1.0], got 1.0570987654320985.
Expected y_max for bbox [0.1475, 0.3472222222222222, 0.4924074074074074, 1.0416666666666665, 'Adina'] to be in the range [0.0, 1.0], got 1.0416666666666665.
Expected y_max for bbox [0.23523140744860832, 0.5548099415204678, 0.5481007671093189, 1.0387280701754384, 'Adina'] to be in the range [0.0, 1.0], got 1.0387280701754384.
Exp

In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join(Image_Split, partition, 'Bibanu')):
        img = cv2.imread(os.path.join(Image_Split, partition, 'Bibanu', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join(Image_Split, partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
                
            width, heigth = img.shape[:2]
            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [width,heigth,width,heigth]))

        try: 
            for x in range(30):
                augmented = augumentation(image=img, bboxes=[coords], class_labels=['Bibanu'])
                cv2.imwrite(os.path.join(aug_data, partition, 'Bibanu', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 2
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join(aug_data, partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

Expected y_max for bbox [0.3530092592592593, 0.611111111111111, 0.607638888888889, 1.0169753086419753, 'Bibanu'] to be in the range [0.0, 1.0], got 1.0169753086419753.
Expected y_max for bbox [0.1863425925925926, 0.29475308641975306, 0.548611111111111, 1.015432098765432, 'Bibanu'] to be in the range [0.0, 1.0], got 1.015432098765432.
Expected y_max for bbox [0.7407407407407408, 0.8395061728395061, 0.3599537037037037, 1.228395061728395, 'Bibanu'] to be in the range [0.0, 1.0], got 1.228395061728395.
Expected y_max for bbox [0.36111111111111105, 0.816358024691358, 0.6759259259259258, 1.1419753086419753, 'Bibanu'] to be in the range [0.0, 1.0], got 1.1419753086419753.
Expected y_max for bbox [0.2175925925925926, 0.7067901234567902, 0.5659722222222222, 1.0416666666666665, 'Bibanu'] to be in the range [0.0, 1.0], got 1.0416666666666665.
x_max is less than or equal to x_min for bbox [0.7465277777777777, 0.32253086419753085, 0.545138888888889, 0.8410493827160493, 'Bibanu'].
Expected y_max for

In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join(Image_Split, partition, 'Papagalu')):
        img = cv2.imread(os.path.join(Image_Split, partition, 'Papagalu', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join(Image_Split, partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
                
            width, heigth = img.shape[:2]
            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [width,heigth,width,heigth]))

        try: 
            for x in range(30):
                augmented = augumentation(image=img, bboxes=[coords], class_labels=['Papagalu'])
                cv2.imwrite(os.path.join(aug_data, partition, 'Papagalu', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 3
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join(aug_data, partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

Expected y_max for bbox [0.43749999999999994, 0.8302469135802468, 0.749375, 1.1721543209876542, 'Papagalu'] to be in the range [0.0, 1.0], got 1.1721543209876542.
Expected y_max for bbox [0.4027777777777778, 0.9027777777777777, 0.5243055555555555, 1.0277777777777777, 'Papagalu'] to be in the range [0.0, 1.0], got 1.0277777777777777.
Expected y_max for bbox [0.4027777777777778, 0.9691358024691357, 0.45601851851851855, 1.0293209876543208, 'Papagalu'] to be in the range [0.0, 1.0], got 1.0293209876543208.
Expected y_max for bbox [0.3032407407407407, 0.8132716049382716, 0.6655092592592592, 1.1419753086419753, 'Papagalu'] to be in the range [0.0, 1.0], got 1.1419753086419753.
Expected y_min for bbox [0.42393518518518514, 1.334934558618769, 0.5539941294387173, 1.5028195488721805, 'Papagalu'] to be in the range [0.0, 1.0], got 1.334934558618769.
Expected y_max for bbox [0.482962962962963, 0.997284878863826, 0.5813425925925926, 1.0982316903369533, 'Papagalu'] to be in the range [0.0, 1.0], got

In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join(Image_Split, partition, 'Pelicanu')):
        img = cv2.imread(os.path.join(Image_Split, partition, 'Pelicanu', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join(Image_Split, partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
                
            width, heigth = img.shape[:2]
            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [width,heigth,width,heigth]))

        try: 
            for x in range(30):
                augmented = augumentation(image=img, bboxes=[coords], class_labels=['Pelicanu'])
                cv2.imwrite(os.path.join(aug_data, partition, 'Pelicanu', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 4
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join(aug_data, partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

Expected y_min for bbox [0.42393518518518514, 1.0494987468671677, 0.5211574074074075, 1.1521860206070729, 'Pelicanu'] to be in the range [0.0, 1.0], got 1.0494987468671677.
Expected y_max for bbox [0.5231481481481481, 0.9598765432098765, 0.716435185185185, 1.1512345679012344, 'Pelicanu'] to be in the range [0.0, 1.0], got 1.1512345679012344.
Expected y_max for bbox [0.4328703703703703, 0.8225308641975307, 0.693287037037037, 1.089506172839506, 'Pelicanu'] to be in the range [0.0, 1.0], got 1.089506172839506.
Expected y_max for bbox [0.39090277777777777, 0.8858199065355373, 0.5726157407407407, 1.0933249633814603, 'Pelicanu'] to be in the range [0.0, 1.0], got 1.0933249633814603.
Expected y_min for bbox [0.22800925925925924, 1.0802469135802468, 0.4386574074074074, 1.2932098765432098, 'Pelicanu'] to be in the range [0.0, 1.0], got 1.0802469135802468.
Expected y_max for bbox [0.4190856481481482, 0.9302648477906209, 0.5475578703703703, 1.1107639973619354, 'Pelicanu'] to be in the range [0.0,

In [6]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']


In [7]:
train_labels = r'/content/drive/MyDrive/AI/Detector/aug_data/labels_train/*.json'
test_labels = r'/content/drive/MyDrive/AI/Detector/aug_data/labels_test/*.json'
val_labels = r'/content/drive/MyDrive/AI/Detector/aug_data/labels_val/*.json'

In [8]:
train_labels = tf.data.Dataset.list_files(train_labels, shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [9]:
test_labels = tf.data.Dataset.list_files(test_labels, shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [10]:
val_labels = tf.data.Dataset.list_files(val_labels, shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [11]:
train_labels.as_numpy_iterator().next()

(array([0], dtype=uint8), array([0., 0., 0., 0.], dtype=float16))

In [ ]:
len(train_labels), len(test_labels), len(val_labels)



(10800, 1590, 2663)

In [ ]:
train_dir = r'/content/drive/MyDrive/AI/Detector/aug_data/train'
test_dir = r'/content/drive/MyDrive/AI/Detector/aug_data/test'
val_dir = r'/content/drive/MyDrive/AI/Detector/aug_data/val'

In [ ]:
len(val_labels)

2663

In [12]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [13]:
train_dir_adina = r'/content/drive/MyDrive/AI/Detector/aug_data/train/Adina/*.jpg'
test_dir_adina = r'/content/drive/MyDrive/AI/Detector/aug_data/test/Adina/*.jpg'
val_dir_adina = r'/content/drive/MyDrive/AI/Detector/aug_data/val/Adina/*.jpg'

train_dir_bibanu = r'/content/drive/MyDrive/AI/Detector/aug_data/train/Bibanu/*.jpg'
test_dir_bibanu = r'/content/drive/MyDrive/AI/Detector/aug_data/test/Bibanu/*.jpg'
val_dir_bibanu = r'/content/drive/MyDrive/AI/Detector/aug_data/val/Bibanu/*.jpg'

train_dir_pelicanu = r'/content/drive/MyDrive/AI/Detector/aug_data/train/Pelicanu/*.jpg'
test_dir_pelicanu = r'/content/drive/MyDrive/AI/Detector/aug_data/test/Pelicanu/*.jpg'
val_dir_pelicanu = r'/content/drive/MyDrive/AI/Detector/aug_data/val/Pelicanu/*.jpg'

train_dir_papagalu = r'/content/drive/MyDrive/AI/Detector/aug_data/train/Papagalu/*.jpg'
test_dir_papagalu = r'/content/drive/MyDrive/AI/Detector/aug_data/test/Papagalu/*.jpg'
val_dir_papagalu = r'/content/drive/MyDrive/AI/Detector/aug_data/val/Papagalu/*.jpg'




In [14]:

train_images_adina = tf.data.Dataset.list_files(train_dir_adina, shuffle=False)
train_images_adina = train_images_adina.map(load_image)
train_images_adina = train_images_adina.map(lambda x: tf.image.resize(x, (120,120)))
train_images_adina = train_images_adina.map(lambda x: x/255)

test_images_adina = tf.data.Dataset.list_files(test_dir_adina, shuffle=False)
test_images_adina = test_images_adina.map(load_image)
test_images_adina = test_images_adina.map(lambda x: tf.image.resize(x, (120,120)))
test_images_adina = test_images_adina.map(lambda x: x/255)

val_images_adina = tf.data.Dataset.list_files(val_dir_adina, shuffle=False)
val_images_adina = val_images_adina.map(load_image)
val_images_adina = val_images_adina.map(lambda x: tf.image.resize(x, (120,120)))
val_images_adina = val_images_adina.map(lambda x: x/255)
                                             

In [15]:
train_images_bibanu = tf.data.Dataset.list_files(train_dir_bibanu, shuffle=False)
train_images_bibanu = train_images_bibanu.map(load_image)
train_images_bibanu = train_images_bibanu.map(lambda x: tf.image.resize(x, (120,120)))
train_images_bibanu = train_images_bibanu.map(lambda x: x/255)

test_images_bibanu = tf.data.Dataset.list_files(test_dir_bibanu, shuffle=False)
test_images_bibanu = test_images_bibanu.map(load_image)
test_images_bibanu = test_images_bibanu.map(lambda x: tf.image.resize(x, (120,120)))
test_images_bibanu = test_images_bibanu.map(lambda x: x/255)

val_images_bibanu = tf.data.Dataset.list_files(val_dir_bibanu, shuffle=False)
val_images_bibanu = val_images_bibanu.map(load_image)
val_images_bibanu = val_images_bibanu.map(lambda x: tf.image.resize(x, (120,120)))
val_images_bibanu = val_images_bibanu.map(lambda x: x/255)

In [16]:
train_images_papagalu = tf.data.Dataset.list_files(train_dir_papagalu, shuffle=False)
train_images_papagalu = train_images_papagalu.map(load_image)
train_images_papagalu = train_images_papagalu.map(lambda x: tf.image.resize(x, (120,120)))
train_images_papagalu = train_images_papagalu.map(lambda x: x/255)

test_images_papagalu = tf.data.Dataset.list_files(test_dir_papagalu, shuffle=False)
test_images_papagalu = test_images_papagalu.map(load_image)
test_images_papagalu = test_images_papagalu.map(lambda x: tf.image.resize(x, (120,120)))
test_images_papagalu = test_images_papagalu.map(lambda x: x/255)

val_images_papagalu = tf.data.Dataset.list_files(val_dir_papagalu, shuffle=False)
val_images_papagalu = val_images_papagalu.map(load_image)
val_images_papagalu = val_images_papagalu.map(lambda x: tf.image.resize(x, (120,120)))
val_images_papagalu = val_images_papagalu.map(lambda x: x/255)

In [17]:
train_images_pelicanu = tf.data.Dataset.list_files(train_dir_pelicanu, shuffle=False)
train_images_pelicanu = train_images_pelicanu.map(load_image)
train_images_pelicanu = train_images_pelicanu.map(lambda x: tf.image.resize(x, (120,120)))
train_images_pelicanu = train_images_pelicanu.map(lambda x: x/255)

test_images_pelicanu = tf.data.Dataset.list_files(test_dir_pelicanu, shuffle=False)
test_images_pelicanu = test_images_pelicanu.map(load_image)
test_images_pelicanu = test_images_pelicanu.map(lambda x: tf.image.resize(x, (120,120)))
test_images_pelicanu = test_images_pelicanu.map(lambda x: x/255)

val_images_pelicanu = tf.data.Dataset.list_files(val_dir_pelicanu, shuffle=False)
val_images_pelicanu = val_images_pelicanu.map(load_image)
val_images_pelicanu = val_images_pelicanu.map(lambda x: tf.image.resize(x, (120,120)))
val_images_pelicanu = val_images_pelicanu.map(lambda x: x/255)

In [18]:
from numpy.matrixlib.defmatrix import concatenate
from tensorflow.python.ops.gen_dataset_ops import concatenate_dataset

train_images = train_images_adina.concatenate(train_images_bibanu).concatenate(train_images_papagalu).concatenate(train_images_pelicanu)
test_images = test_images_adina.concatenate(test_images_bibanu).concatenate(test_images_papagalu).concatenate(test_images_pelicanu)
val_images = val_images_adina.concatenate(val_images_bibanu).concatenate(val_images_papagalu).concatenate(val_images_pelicanu)



In [19]:
train_dataset = tf.data.Dataset.zip((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

In [20]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 120, 120, None), dtype=tf.float32, name=None), (TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), TensorSpec(shape=<unknown>, dtype=tf.float16, name=None)))>

In [21]:

test_dataset = tf.data.Dataset.zip((test_images, test_labels))
test_dataset = test_dataset.batch(8).prefetch(tf.data.AUTOTUNE)

In [22]:
val = tf.data.Dataset.zip((val_images, val_labels))
val_dataset = val.shuffle(buffer_size=1000).batch(8).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_dataset

<ShuffleDataset element_spec=(TensorSpec(shape=(None, 120, 120, None), dtype=tf.float32, name=None), (TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), TensorSpec(shape=<unknown>, dtype=tf.float16, name=None)))>

In [ ]:
test_dataset

<ShuffleDataset element_spec=(TensorSpec(shape=(None, 120, 120, None), dtype=tf.float32, name=None), (TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), TensorSpec(shape=<unknown>, dtype=tf.float16, name=None)))>

In [ ]:
train_dataset.element_spec


(TensorSpec(shape=(None, 120, 120, None), dtype=tf.float32, name=None),
 (TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None),
  TensorSpec(shape=<unknown>, dtype=tf.float16, name=None)))

In [ ]:
val_dataset.element_spec

(TensorSpec(shape=(None, 120, 120, None), dtype=tf.float32, name=None),
 (TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None),
  TensorSpec(shape=<unknown>, dtype=tf.float16, name=None)))

In [ ]:
train_dataset.as_numpy_iterator().next()[0].shape

In [ ]:
train_labels.as_numpy_iterator().next()[0].shape

(1,)

In [ ]:
train_labels.as_numpy_iterator().next()

(array([0], dtype=uint8), array([0., 0., 0., 0.], dtype=float16))

In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [24]:
vgg = VGG16(include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:

vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [25]:
def build_model(): 
    input_layer = Input(shape=(120,120,3))
    
    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model  
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(4, activation='softmax')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    
    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

In [26]:
facetracker = build_model()
facetracker.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, None, None,   14714688    ['input_2[0][0]']                
                                512)                                                              
                                                                                                  
 global_max_pooling2d (GlobalMa  (None, 512)         0           ['vgg16[0][0]']                  
 xPooling2D)                                                                                  

In [27]:
batches_per_epoch = len(train_dataset)
lr_decay = (1./0.75 -1)/batches_per_epoch

In [28]:

opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)

In [29]:
def localization_loss(y_true, yhat):            
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))
                  
    h_true = y_true[:,3] - y_true[:,1] 
    w_true = y_true[:,2] - y_true[:,0] 

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord + delta_size

In [30]:
classloss = tf.keras.losses.CategoricalCrossentropy()
regressloss = localization_loss

In [31]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)


In [32]:
model = FaceTracker(facetracker)

In [33]:
model.compile(opt, classloss, regressloss)

In [34]:
logdir='logs'

In [35]:

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:

hist = model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[tensorboard_callback])

Epoch 1/10
124/338 [==========>...................] - ETA: 24:38 - total_loss: 11.1638 - class_loss: 4.2172 - regress_loss: 9.0553